In [ ]:
import pandas as pd
from tabulate import tabulate

In [ ]:
df = pd.read_csv('C:/Users/Thalisson/Documents/GitHub/inaFaceAnalyzer/output/catraca1M_20F2D.csv')
print(tabulate(df, headers='keys', tablefmt='pretty'))

# frame: o caminho completo para a imagem usada
# bbox: (esquerda, topo, direita, baixo) a caixa delimitadora do rosto no quadro da imagem
# detect_conf: a estimativa de confiança da detecção de rosto (dependendo do método de detecção de rosto utilizado)
# sex_decfunc: saída bruta do classificador de gênero: valores positivos são usados para homens e valores negativos para mulheres
# sex_label: predição do classificador de gênero: 'm' para homens e 'w' para mulheres
# age_decfunc: saída bruta da regressão de idade baseada nas categorias de idade do FairFace.
# 0 para (0-3 anos), 1 para (4-9 anos), 2 para (10-19 anos), 3 para (20-29 anos), etc...
# age_label: predição de idade "human readable"

In [ ]:
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # area de inters
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    
    # area de 2 bbox
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    # area de uniao
    unionArea = boxAArea + boxBArea - interArea
    
    iou = interArea / float(unionArea)
    return iou

# Sem geração de vídeo

In [ ]:
df = df.drop_duplicates(subset=['face_id'], keep='first')
df.reset_index(drop=True, inplace=True)
print(tabulate(df, headers='keys', tablefmt='pretty'))

# frame: o caminho completo para a imagem usada
# bbox: (esquerda, topo, direita, baixo) a caixa delimitadora do rosto no quadro da imagem
# detect_conf: a estimativa de confiança da detecção de rosto (dependendo do método de detecção de rosto utilizado)
# sex_decfunc: saída bruta do classificador de gênero: valores positivos são usados para homens e valores negativos para mulheres
# sex_label: predição do classificador de gênero: 'm' para homens e 'w' para mulheres
# age_decfunc: saída bruta da regressão de idade baseada nas categorias de idade do FairFace.
# 0 para (0-3 anos), 1 para (4-9 anos), 2 para (10-19 anos), 3 para (20-29 anos), etc...
# age_label: predição de idade "human readable"

In [ ]:
iou_threshold = 0.3

# coluna para verificar os ids de face q sao iguais
df['same_face_id'] = -1
ious = []

for i in range(len(df)):
    biggestIou = 0
    current_face_id = df.loc[i, 'face_id']
    print(current_face_id)
    for j in range(i + 1, len(df)):
        boxA = eval(df.loc[i, 'bbox'])
        boxB = eval(df.loc[j, 'bbox'])
        
        if df.loc[j, 'face_id'] != current_face_id:
            iou = calculate_iou(boxA, boxB)
            ious.append(iou)
            if iou > biggestIou:
                biggestIou = iou
                best_match_id = df.loc[j, 'face_id']
            # print(f'iou: {iou} {current_face_id}')

    print(biggestIou)
    if biggestIou > iou_threshold:
        if df.loc[i, 'same_face_id'] == -1 and df.loc[df['face_id'] == best_match_id, 'same_face_id'].iloc[0] == -1:
            same_face_id = df.loc[i, 'face_id']
            df.loc[i, 'same_face_id'] = same_face_id
            df.loc[df['face_id'] == best_match_id, 'same_face_id'] = same_face_id
        elif df.loc[i, 'same_face_id'] != -1:
            same_face_id = df.loc[i, 'same_face_id']
            df.loc[df['face_id'] == best_match_id, 'same_face_id'] = same_face_id
        elif df.loc[df['face_id'] == best_match_id, 'same_face_id'].iloc[0] != -1:
            same_face_id = df.loc[df['face_id'] == best_match_id, 'same_face_id'].iloc[0]
            df.loc[i, 'same_face_id'] = same_face_id

# se ainda for -1, nao teve nenhuma correspodencia, entao setamos com o proprio id.
df.loc[df['same_face_id'] == -1, 'same_face_id'] = df['face_id']



In [ ]:
print(tabulate(df, headers='keys', tablefmt='pretty'))
# print(ious)


# filtered_df = df[df['same_face_id'] == 0]
# print(tabulate(filtered_df, headers='keys', tablefmt='pretty'))

In [ ]:
grouped_df = df.groupby('same_face_id').agg({
    'sex_decfunc_avg': 'mean',  # media do sex_decfunc_avg
    'age_label_avg': 'mean'   # media do age_decfunc_avg
}).reset_index()

print(tabulate(grouped_df, headers='keys', tablefmt='pretty'))
# grouped_df.to_csv('C:/Users/Thalisson/Documents/GitHub/inaFaceAnalyzer/output/catraca1M_20F2D_iou02.csv', index=False)

# df = pd.read_csv('C:/Users/Thalisson/Documents/GitHub/inaFaceAnalyzer/output/catraca1M_20F1D_iou02.csv')
# print(tabulate(df, headers='keys', tablefmt='pretty'))

# Geração de vídeo

In [ ]:
# iou_threshold = 0.2

# # coluna para verificar os ids de face q sao iguais
# df['same_face_id'] = -1
# ious = []
# for i in range(len(df)):
#     for j in range(i + 1, len(df)):
#         boxA = eval(df.loc[i, 'bbox'])
#         boxB = eval(df.loc[j, 'bbox'])
#         iou = calculate_iou(boxA, boxB)
#         ious.append(iou)
#         if iou > iou_threshold:
#             if df.loc[i, 'same_face_id'] == -1 and df.loc[j, 'same_face_id'] == -1:
#                 df.loc[i, 'same_face_id'] = df.loc[i, 'face_id']
#                 df.loc[j, 'same_face_id'] = df.loc[i, 'face_id']
#             elif df.loc[i, 'same_face_id'] != -1:
#                 df.loc[j, 'same_face_id'] = df.loc[i, 'same_face_id']
#             elif df.loc[j, 'same_face_id'] != -1:
#                 df.loc[i, 'same_face_id'] = df.loc[j, 'same_face_id']

# # se ainda for -1, nao teve nenhuma correspodencia, entao setamos com o proprio id.
# df.loc[df['same_face_id'] == -1, 'same_face_id'] = df['face_id']



In [ ]:
# COMPARATIVO INDEX V1

iou_threshold = 0.3

# PEGAR A ULTIMA APARIÇAO DO FACE ID > I, PO REXEMPLO ULTIMA APARIÇAO DA BOX COM FACEID 0 E COMPARAR COM A PRIMEIRA APARIÇÃO DOS OUTROS FACE IDS

df['same_face_id'] = -1
face_to_same_face = {}


for i in range(len(df)):
    biggestIou = 0
    current_face_id = df.loc[i, 'face_id']
    print(f'face id: {current_face_id}')
    
    # se o id já tiver passado pelo iou (evitar que faces detectadas como iguais pelo modelo passem pelo cálculo)
    if current_face_id in face_to_same_face:
        df.loc[i, 'same_face_id'] = face_to_same_face[current_face_id]
        print(f'pulando face id {current_face_id}, ja processado com same_face_id {face_to_same_face[current_face_id]}')
        continue
    
    for j in range(i + 1, len(df)):
        boxA = eval(df.loc[i, 'bbox'])
        boxB = eval(df.loc[j, 'bbox'])

        if df.loc[j, 'face_id'] != current_face_id:
            iou = calculate_iou(boxA, boxB)
            if iou > biggestIou:
                biggestIou = iou
                best_match_id = j
            # print(f'iou: {iou} {current_face_id}')

    print(biggestIou)
    if biggestIou > iou_threshold:
        print(f'melhor id foi: {best_match_id}')
        if df.loc[i, 'same_face_id'] == -1 and df.loc[df['face_id'] == best_match_id, 'same_face_id'].iloc[0] == -1:
            same_face_id = df.loc[i, 'face_id']
            df.loc[i, 'same_face_id'] = same_face_id
            df.loc[df['face_id'] == best_match_id, 'same_face_id'] = same_face_id
            face_to_same_face[current_face_id] = same_face_id
            face_to_same_face[best_match_id] = same_face_id
        elif df.loc[i, 'same_face_id'] != -1:
            same_face_id = df.loc[i, 'same_face_id']
            df.loc[df['face_id'] == best_match_id, 'same_face_id'] = same_face_id
            face_to_same_face[best_match_id] = same_face_id
        elif df.loc[df['face_id'] == best_match_id, 'same_face_id'].iloc[0] != -1:
            same_face_id = df.loc[df['face_id'] == best_match_id, 'same_face_id'].iloc[0]
            df.loc[i, 'same_face_id'] = same_face_id
            face_to_same_face[current_face_id] = same_face_id

df.loc[df['same_face_id'] == -1, 'same_face_id'] = df['face_id']


In [ ]:
# V2
iou_threshold = 0.3

df['same_face_id'] = -1
face_to_same_face = {}

unique_face_ids = df['face_id'].unique()

for face_id in unique_face_ids:
    i = df[df['face_id'] == face_id].index[-1]  # Pegando a última ocorrência do face_id
    biggestIou = 0
    print(f'face id: {face_id}')
    
    if face_id in face_to_same_face:
        df.loc[df['face_id'] == face_id, 'same_face_id'] = face_to_same_face[face_id]
        print(f'pulando face id {face_id}, já processado com same_face_id {face_to_same_face[face_id]}')
        continue
    
    for other_face_id in unique_face_ids:
        if other_face_id != face_id:
            j = df[df['face_id'] == other_face_id].index[0] 
            boxA = eval(df.loc[i, 'bbox'])
            boxB = eval(df.loc[j, 'bbox'])

            iou = calculate_iou(boxA, boxB)
            if iou > biggestIou:
                biggestIou = iou
                best_match_id = other_face_id

    print(biggestIou)
    if biggestIou > iou_threshold:
        print(f'melhor face id foi: {best_match_id}')
        if df.loc[i, 'same_face_id'] == -1 and df.loc[df['face_id'] == best_match_id, 'same_face_id'].iloc[0] == -1:
            same_face_id = face_id
            df.loc[df['face_id'] == face_id, 'same_face_id'] = same_face_id
            df.loc[df['face_id'] == best_match_id, 'same_face_id'] = same_face_id
            face_to_same_face[face_id] = same_face_id
            face_to_same_face[best_match_id] = same_face_id
        elif df.loc[i, 'same_face_id'] != -1:
            same_face_id = df.loc[i, 'same_face_id']
            df.loc[df['face_id'] == best_match_id, 'same_face_id'] = same_face_id
            face_to_same_face[best_match_id] = same_face_id
        elif df.loc[df['face_id'] == best_match_id, 'same_face_id'].iloc[0] != -1:
            same_face_id = df.loc[df['face_id'] == best_match_id, 'same_face_id'].iloc[0]
            df.loc[df['face_id'] == face_id, 'same_face_id'] = same_face_id
            face_to_same_face[face_id] = same_face_id

df.loc[df['same_face_id'] == -1, 'same_face_id'] = df['face_id']

In [ ]:
#V3
iou_threshold = 0.1
df['same_face_id'] = -1
face_to_same_face = {}

# Iterar por todos os face_ids únicos
unique_face_ids = df['face_id'].unique()
max_frame = df['frame'].max()
min_frame = df['frame'].min()

for face_id in unique_face_ids:
    face_indices = df[df['face_id'] == face_id].index
    current_frame = df.loc[face_indices[-1], 'frame']
    print(f'current frame: {current_frame}')
    biggestIou = 0
    best_match_id = None
    
    if face_id in face_to_same_face:
        print(f"face ID {face_id} nao vai ser processado, ja ta no dicionário com same_face_id {face_to_same_face[face_id]}")
        df.loc[face_indices, 'same_face_id'] = face_to_same_face[face_id]
        continue

    print(f"processando face ID {face_id}")
    
    # verificar 45 frames para frente
    forward_ids = []
    for frame in range(current_frame + 1, min(current_frame + 46, max_frame + 1)):
        # print(f'comparando frame: {frame}')
        forward_indices = df[df['frame'] == frame].index
        for idx in forward_indices:
            if df.loc[idx, 'face_id'] != face_id:  # face id diferente
                forward_ids.append(df.loc[idx, 'face_id'])
                boxA = eval(df.loc[face_indices[-1], 'bbox'])  # ultima apariçao do id atual
                boxB = eval(df.loc[idx, 'bbox'])
                iou = calculate_iou(boxA, boxB)
                if iou > biggestIou:
                    biggestIou = iou
                    best_match_id = df.loc[idx, 'face_id']
                    best_match_index = idx

    print(f"ids nos 45 frames pra frente: {face_id}: {forward_ids}")

    # verificar 45 frames para trás
    backward_ids = []
    for frame in range(current_frame - 1, max(current_frame - 46, min_frame - 1), -1):
        # print(f'comparando frame: {frame}')
        backward_indices = df[df['frame'] == frame].index
        for idx in backward_indices:
            if df.loc[idx, 'face_id'] != face_id:  # id diferente
                backward_ids.append(df.loc[idx, 'face_id'])
                boxA = eval(df.loc[face_indices[0], 'bbox'])  # primeira apariçao
                boxB = eval(df.loc[idx, 'bbox'])
                iou = calculate_iou(boxA, boxB)
                if iou > biggestIou:
                    biggestIou = iou
                    best_match_id = df.loc[idx, 'face_id']
                    best_match_index = idx

    print(f"ids nos 45 frames para tras: {face_id}: {backward_ids}")

    if best_match_id is not None and biggestIou > iou_threshold:
        print(f"melhor correspondência para o face ID {face_id} foi face ID {best_match_id} com o IOU de {biggestIou}")
        if df.loc[face_indices[-1], 'same_face_id'] == -1 and df.loc[best_match_index, 'same_face_id'] == -1:
            same_face_id = min(face_id, best_match_id)
            df.loc[df['face_id'] == face_id, 'same_face_id'] = same_face_id
            df.loc[df['face_id'] == best_match_id, 'same_face_id'] = same_face_id
            face_to_same_face[face_id] = same_face_id
            face_to_same_face[best_match_id] = same_face_id
        elif df.loc[face_indices[-1], 'same_face_id'] != -1:
            same_face_id = df.loc[face_indices[-1], 'same_face_id']
            df.loc[df['face_id'] == best_match_id, 'same_face_id'] = same_face_id
            face_to_same_face[best_match_id] = same_face_id
        elif df.loc[best_match_index, 'same_face_id'] != -1:
            same_face_id = df.loc[best_match_index, 'same_face_id']
            df.loc[df['face_id'] == face_id, 'same_face_id'] = same_face_id
            face_to_same_face[face_id] = same_face_id

df.loc[df['same_face_id'] == -1, 'same_face_id'] = df['face_id']

In [ ]:
iou_threshold = 0.1
df['same_face_id'] = -1
face_to_same_face = {}

# atualizar o same_face_id para todos os face_ids relacionados
def update_same_face_id(face_id, new_same_face_id):
    current_same_face_id = face_to_same_face.get(face_id, face_id)
    for key, value in face_to_same_face.items():
        if value == current_same_face_id:
            face_to_same_face[key] = new_same_face_id
            print(f"Atualizado face ID {key} para same_face_id {new_same_face_id}")

# propagar o mesmo same_face_id para todos os vinculados
def propagate_same_face_id():
    for face_id in face_to_same_face:
        same_face_id = face_to_same_face[face_id]
        df.loc[df['face_id'] == face_id, 'same_face_id'] = same_face_id

# iterar face ids unicos (evitar repetiçao)
unique_face_ids = df['face_id'].unique()
max_frame = df['frame'].max()
min_frame = df['frame'].min()

for face_id in unique_face_ids:
    face_indices = df[df['face_id'] == face_id].index
    current_frame = df.loc[face_indices[-1], 'frame']
    print(f'current frame: {current_frame}')
    biggestIou = 0
    best_match_id = None
    
    if face_id in face_to_same_face:
        print(f"face ID {face_id} nao vai ser processado, ja ta no dicionário com same_face_id {face_to_same_face[face_id]}")
        df.loc[face_indices, 'same_face_id'] = face_to_same_face[face_id]
        continue

    print(f"processando face ID {face_id}")
    
    # verificar 45 frames para frente
    forward_ids = []
    for frame in range(current_frame + 1, min(current_frame + 46, max_frame + 1)):
        forward_indices = df[df['frame'] == frame].index
        for idx in forward_indices:
            if df.loc[idx, 'face_id'] != face_id:  # face id diferente
                forward_ids.append(df.loc[idx, 'face_id'])
                boxA = eval(df.loc[face_indices[-1], 'bbox'])  # ultima apariçao do id atual
                boxB = eval(df.loc[idx, 'bbox'])
                iou = calculate_iou(boxA, boxB)
                if iou > biggestIou:
                    biggestIou = iou
                    best_match_id = df.loc[idx, 'face_id']
                    best_match_index = idx

    print(f"ids nos 45 frames pra frente: {face_id}: {forward_ids}")

    # verificar 45 frames para trás
    backward_ids = []
    for frame in range(current_frame - 1, max(current_frame - 46, min_frame - 1), -1):
        backward_indices = df[df['frame'] == frame].index
        for idx in backward_indices:
            if df.loc[idx, 'face_id'] != face_id:  # id diferente
                backward_ids.append(df.loc[idx, 'face_id'])
                boxA = eval(df.loc[face_indices[0], 'bbox'])  # primeira apariçao
                boxB = eval(df.loc[idx, 'bbox'])
                iou = calculate_iou(boxA, boxB)
                if iou > biggestIou:
                    biggestIou = iou
                    best_match_id = df.loc[idx, 'face_id']
                    best_match_index = idx

    print(f"ids nos 45 frames para tras: {face_id}: {backward_ids}")

    if best_match_id is not None and biggestIou > iou_threshold:
        print(f"melhor correspondência para o face ID {face_id} foi face ID {best_match_id} com o IOU de {biggestIou}")
        if df.loc[face_indices[-1], 'same_face_id'] == -1 and df.loc[best_match_index, 'same_face_id'] == -1:
            same_face_id = min(face_id, best_match_id)
            df.loc[df['face_id'] == face_id, 'same_face_id'] = same_face_id
            df.loc[df['face_id'] == best_match_id, 'same_face_id'] = same_face_id
            face_to_same_face[face_id] = same_face_id
            face_to_same_face[best_match_id] = same_face_id
        elif df.loc[face_indices[-1], 'same_face_id'] != -1:
            same_face_id = df.loc[face_indices[-1], 'same_face_id']
            df.loc[df['face_id'] == best_match_id, 'same_face_id'] = same_face_id
            face_to_same_face[best_match_id] = same_face_id
        elif df.loc[best_match_index, 'same_face_id'] != -1:
            same_face_id = df.loc[best_match_index, 'same_face_id']
            df.loc[df['face_id'] == face_id, 'same_face_id'] = same_face_id
            face_to_same_face[face_id] = same_face_id

        # atualizar todos os same_face_ids relacionados
        update_same_face_id(face_id, same_face_id)
        update_same_face_id(best_match_id, same_face_id)

    # passar o samefaceid a cada iteraçao
    propagate_same_face_id()

# same_face_id fica face_id para casos que ficaram com -1 (sem correspodencnia)
df.loc[df['same_face_id'] == -1, 'same_face_id'] = df['face_id']

In [ ]:
print(tabulate(df, headers='keys', tablefmt='pretty'))
df.to_csv('C:/Users/Thalisson/Documents/GitHub/inaFaceAnalyzer/output/catracaTesteV3_1.csv', index=False)

In [ ]:
filtered_df = df[df['same_face_id'] == 71
                 ]
print(tabulate(filtered_df, headers='keys', tablefmt='pretty'))

In [ ]:
df2 = df.drop_duplicates(subset=['same_face_id'], keep='first')
grouped_df = df.groupby('same_face_id').agg({
    'sex_decfunc_avg': 'mean',  # média do sex_decfunc_avg
    'age_label_avg': 'mean'     # média do age_decfunc_avg
}).reset_index()

print(tabulate(grouped_df, headers='keys', tablefmt='pretty'))
grouped_df.to_csv('C:/Users/Thalisson/Documents/GitHub/inaFaceAnalyzer/output/catracaTesteV3.csv', index=False)

In [ ]:
import cv2
import pandas as pd
import ast

def draw_bounding_boxes(frame, df, frame_index):
    rows = df[df['frame'] == frame_index]
    for _, row in rows.iterrows():
        bbox = ast.literal_eval(row['bbox'])
        same_face_id = row['same_face_id']
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
        
        # id face
        cv2.putText(frame, f'ID: {same_face_id}', (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)



video_path = 'C:/Users/Thalisson/Documents/GitHub/inaFaceAnalyzer/media/catraca1M_20F1D.avi'
cap = cv2.VideoCapture(video_path)

output_video = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

frame_index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    draw_bounding_boxes(frame, df, frame_index)
    out.write(frame)
    frame_index += 1

cap.release()
out.release()
cv2.destroyAllWindows()
